In [ ]:
import pandas as pd
import numpy as np

from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import word_tokenize

# Keras
from tensorflow.keras.preprocessing.text import Tokenizer

fp = open("Sarcasm_tweets.txt", 'r')
id_tweet_map = {}
tweet_id_map = {}
count = 0
for line in fp:
  line = line.strip()
  tokens = line.split(' ')
  if len(tokens) == 1 and tokens[0] != '':
    current_id = tokens[0]
  elif len(tokens) > 1:
    id_tweet_map[current_id] = line

import string
fp = open("Sarcasm_tweet_truth.txt", 'r')
id_truth_map = {}

for line in fp:
  line = line.strip()
  if line == '':
    continue
  elif line[0] in string.digits:
    current_id = line
  else:
    id_truth_map[current_id] = line

id_tweet = pd.DataFrame(id_tweet_map.items(), columns=['tweet_id', 'tweet'])
lbl_tweet = pd.DataFrame(id_truth_map.items(), columns=['tweet_id', 'sarcasm'])
dataset = pd.merge(id_tweet, lbl_tweet, on='tweet_id')
dataset = dataset.replace({'sarcasm': {'YES': 1, 'NO': 0}})

tweet_rm=[]
for j in range(0,len(dataset)):
  tweet_rm.append(dataset['tweet'].iloc[j].replace('#sarcasm','').replace('#irony','').replace('#Irony','').replace('#Sarcasm',''))
dataset['tweet'] = tweet_rm



## Add pre-processing ##
import emoji
import re

#not needed now
FLAGS = re.MULTILINE | re.DOTALL
def emotize(text):
    t1 = emoji.replace_emoji(text,' emoji ')
    return t1
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}
def contraction_map(text):
    newstring = text
    newstring = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newstring.split(" ")])
    return "".join(str(newstring)).strip()

def text_preprocess(text, row_dict):
  dict_row = row_dict
  tknzr = WhitespaceTokenizer()
  FLAGS = re.MULTILINE | re.DOTALL
  # Different regex parts for smiley faces
  eyes = r"[8:=;]"
  nose = r"['`\-]?"
  emoticon_mapping = {
    ":)": "smile",
    ":(": "sad",
    ":D": "laugh",
    ":P": "tongue",
    ";)": "wink",
    "<3": "heart",
    ":/": "skeptical",
    ":|": "neutral",
    ":O": "surprised",
    ":*": "kiss",
    ":')": "happy tears",
    ":/": "unsure",
    ":3": "cute",
    ":'(": "crying"
    }
  slangs_mapping = {
      "RT":	"Retweet",
      "OH":	"Overheard",
      "FF":	"Follow Friday",
      "HT": "Heard Through",
      "MM":	"Music Monday",
      "DM":	"Direct Message",
      "Tweetup":	"A real life meetup announced on twitter",
      "SM":	"Social Media",
      "SB":	"Small Business",
      "ICYMI":	"In Case You Missed It",
      "MRT":	"Modified ReTweet",
      "MT":	"Modified Tweet",
      "NTS":	"Note To Self",
      "NP":	"Now Playing",
      "CC":	"Carbon Copy",
      'BFFL': 'Best friends for life'
  }

  # function so code less repetitive
  def re_sub(pattern, repl):
      return re.sub(pattern, repl, text, flags=FLAGS)
  text = emotize(text)

  def replace_emoticons(text):
    pattern = re.compile(r'(:\)|:\(|:D|:P|;\)|<3)')
    return pattern.sub(lambda match: emoticon_mapping[match.group()], text)
  text = replace_emoticons(text)

  def replace_slangs(text):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in slangs_mapping.keys()) + r')\b')
    return pattern.sub(lambda x: slangs_mapping[x.group()], text)
  text = replace_slangs(text)

  text = contraction_map(text)

  if dict_row['hashtag']==1:
    text = re.sub(r'#', '', text)# segment hastags

  if dict_row['userhandles'] == 1:
    text = re_sub(r"@\w+", "user") #replace username to 'user
  else:
    text = re_sub(r"@\w+", " ") #remove username

  if dict_row['hyperlinks'] == 1:
    text = re_sub(r"r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "url")
  else:
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "")

  def reduce_elongated_characters(text):
    pattern = re.compile(r'(\w+)(\W)\2*')
    return pattern.sub(r'\1\2', text)
  if dict_row['elongated_chararcters']==1:
    text = reduce_elongated_characters(text)

  def reduce_elongated_words(text):
    pattern = re.compile(r'(\w)(\1{2,})(\W*)')
    return pattern.sub(r'\1\1\3', text)

  def add_intense_before_elongated_words(text):
    pattern = re.compile(r'(\b\w*(\w)\2{2,}\w*\b)')
    return pattern.sub(r'intense \1', text)
  def intense_elongated_words(text):
    text = add_intense_before_elongated_words(text)
    pattern = re.compile(r'(\w)(\1{2,})(\W*)')
    return pattern.sub(r'\1\1\3', text)

  if dict_row['replace_elongated_words'] == 0:
    text = reduce_elongated_words(text)
  else:
    text = intense_elongated_words(text)

  alphnumeric_slangs = {
    '2nite': 'Tonight',
    '4eva': 'Forever',
    'L8r': 'Later',
    'B4': 'Before',
    '2moro': 'Tomorrow',
    '2day': 'Today',
    'G2g': 'Got to go',
    'Gr8': 'Great',
    '411': 'Information or news',
    '2b': 'To be',
    '24/7': 'All the time',
    '1derful': 'Wonderful',
    '2gether': 'Together',
    '4get': 'Forget'
    }
  def replace_alpha_slangs(text):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in alphnumeric_slangs.keys()) + r')\b')
    return pattern.sub(lambda x: alphnumeric_slangs[x.group()], text)

  if row_dict['alphanumeric_slangs'] == 1:
    text = replace_alpha_slangs(text)

  def remove_punctuations(text):
    pattern = re.compile(r'[^\w\s]')
    return pattern.sub('', text)
  if row_dict['remove_punctuations'] == 1:
    text = remove_punctuations(text)

  def add_intense_to_uppercase_words(text):
    pattern = re.compile(r'\b([A-Z]+)\b')
    return pattern.sub(r'intense \1', text)
  if row_dict['capitalized_words'] == 1:
    text = add_intense_to_uppercase_words(text)

  def split_mix_case_words(text):
    processed_text = ""
    prev_char = ""
    middle_char = ""

    for char in text:
        if char.isupper() and middle_char.islower():
            processed_text += " "
        if prev_char.isupper() and middle_char.isupper() and char.islower():
            processed_text = processed_text[:-1]
            processed_text += " "
            processed_text += middle_char
        if char.isdigit() and middle_char.islower() and not middle_char.isdigit():
            processed_text += " "
        if char.isupper() and middle_char.isdigit():
            processed_text += " "
        if char.islower() and middle_char.isdigit():
            processed_text += " "
        processed_text += char
        prev_char = middle_char
        middle_char = char

    return processed_text
  if row_dict['Split_MixCase'] == 1:
    text = split_mix_case_words(text)

  def split_mix_script(text):
    pattern = re.compile(r'(?<=[a-zA-Z])(?=[^a-zA-Z])|(?<=[^a-zA-Z])(?=[a-zA-Z])')
    processed_text = pattern.sub(r' ', text)
    return processed_text
  if row_dict['Split_MixScript'] == 1:
    text = split_mix_script(text)

  def replace_recurrent_words(text):
    prev_word = ""
    more_prev_word = ""
    processed_word = ""
    words = list(text.split())
    new_words = []
    intial=0
    reccur=0
    count=0
    while intial < (len(words)-1):
      count+=1
      if count == 2000:
          break
      if intial < (len(words)-2):
          if words[intial] == words[intial + 1] == words[intial+2] and words[intial]!='user' and words[intial]!='url':
              reccur=intial
              while reccur < (len(words)-1):
                if words[intial] == words[reccur]:
                  reccur+=1
                else:
                  #print(j)
                  break
              new_words.append("intense")
              new_words.append(words[intial])
              intial=reccur
          else:
            new_words.append(words[intial])
            intial+=1
      else:
        new_words.append(words[intial])
        #new_words.append(word[intial+1])
        intial+=1

    if reccur != len(words) - 1:
      new_words.append(words[intial])
    return " ".join(new_words)

  if row_dict['recurrent_word'] == 1:
    text = replace_recurrent_words(text)

  #tokens = tknzr(text.lower())
  tokens = tknzr.tokenize(text)
  return " ".join(tokens)

row_dict={'hashtag':1,
 'userhandles':0,
 'hyperlinks':1,
 'emoji':1,
 'emoticons':1,
 'word_slangs':1,
 'expanding_contraction':1,
 'elongated_chararcters':0,
 'replace_elongated_words':1,
 'alphanumeric_slangs':1,
 'remove_punctuations':0,
 'capitalized_words':0,
 'recurrent_word':0,
 'Split_MixCase':1,
 'Split_MixScript':1,
 'misspled_words':0,
 'stopwords_removal':1}

for line in range (0,len(dataset)):
  sent = dataset['tweet'].iloc[line]
  sent = text_preprocess(sent, row_dict) #what to process
  dataset.loc[line, 'processed_tweets'] = sent

In [ ]:
dataset.head()

,tweet_id,tweet,sarcasm,processed_tweets
0,866871160725794816,Triple Talaq par Burbak Kuchh nahi bolega,0,Triple Talaq par Burbak Kuchh nahi bolega
1,880356789358743553,Batao ye uss site pr se akki sir ke verdict ni...,1,Batao ye uss site pr se akki sir ke verdict ni...
2,877751493889105920,Hindu baheno par julam bardas nahi hoga @Tripl...,0,Hindu baheno par julam bardas nahi hoga Hindu ...
3,901806457871466496,Naa bhai.. aisa nhi hai.. mere handle karne se...,0,Naa bhai .. aisa nhi hai .. mere handle karne ...
4,866264330748219392,#RememberingRajiv aaj agar musalman auraten tr...,0,Remembering Rajiv aaj agar musalman auraten tr...


In [ ]:
dataset['processed_tweets'].iloc[2]


'Hindu baheno par julam bardas nahi hoga Hindu daram par lago hoga hamari Hindu baheno ki soraksa ke liye'

In [ ]:
dataset['tweet'].iloc[2]

'Hindu baheno par julam bardas nahi hoga @TripleTalaq Hindu daram par lago hoga hamari Hindu baheno ki soraksa ke liye'

In [ ]:
!pip install tensorflow

In [ ]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 6.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive
